# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

# Tutorial 3: Datos vectoriales II.


Este tutorial tiene como objetivos:
- Construir geometrías envolventes como la envoltura convexa y el rectángulo envolvente.
- Calcular distancias entre geometrías y entre geoseries.
- Generar mapas interactivos.

### Contexto: 

En diciembre de 2022, Viña del Mar sufrió un [grave incendio urbano-forestal](https://www.bbc.com/mundo/noticias-america-latina-64073289) que comenzó en una zona de vegetación cercana a Rodelillo y, debido al viento intenso y a las condiciones de sequedad, avanzó rápidamente hacia sectores habitados. El fuego destruyó numerosas viviendas, dejó cientos de damnificados y provocó la muerte de dos personas, generando una emergencia en plena víspera de Navidad. Las autoridades declararon alerta roja y estado de catástrofe, movilizando a brigadas, bomberos y equipos de apoyo para contener el siniestro y asistir a las familias afectadas. El incendio evidenció la alta vulnerabilidad de algunos asentamientos y la necesidad de fortalecer la planificación territorial y la prevención de incendios forestales en la región.


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import contextily as ctx

## 2. Datos.

A  nuestros datos anteriores
- `vm_comuna.gpkg`
- `vm_manz.gpkg`

Agregamos dos nuevas capas:

-`Edificaciones.shp`: edificaciones catastradas luego del gran incendio de diciembre 2022. Por cada vivienda catastrada, se incluye la huella de edificación, y una clasificación del tito de daño ocasionado por el incendio: *Ninguno*, *Parcial*, o *Total*.

-`grifos_nacionales_2022.shp`: listado de grifos existentes en Chile, de acuerdo a la información de la Superintendencia de Servicios Sanitarios.


## 3. Ejercicios.



### 3.1 Lectura e inspección de datos vectoriales.

In [ ]:
vina=gpd.read_file('vm_comuna.gpkg')
manz=gpd.read_file('vm_manz.gpkg')


In [ ]:
grifos=gpd.read_file('data_T3/Grifos/grifos_nacionales_2022.shp').to_crs(32719)
grifos.plot()

### 3.2 Selección de viviendas dañadas

In [ ]:
edif=gpd.read_file('data_T3/Edificaciones.shp').to_crs(32719)
damaged=edif[edif['N_Daño']!='Ninguno']

In [ ]:
damaged.plot()

### 3.3 Envoltura, envoltura convexa y clip.

Para identificar el área afectada por el incendio, podemos usar la evoltura convexa o el rectángulo envolvente de las viviendas dañadas. Para ello es encesario unir todas las geometrías (con [union_all](https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoSeries.union_all.html) o [unary_union](https://geopandas.org/en/v0.9.0/docs/reference/api/geopandas.GeoSeries.unary_union.html) ), y luego calcular la envolvente con [convex_hull](https://geopandas.org/en/v1.1.1/docs/reference/api/geopandas.GeoSeries.convex_hull.html) o [envelope](https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoSeries.envelope.html). Es conveniente usar la geometría resultante para crear un nuevo `GeoDataFrame`.


In [ ]:
cv=gpd.GeoDataFrame(geometry=[damaged.union_all().convex_hull],crs=edif.crs) #envoltura convexa
env=gpd.GeoDataFrame(geometry=[edif.union_all().envelope],crs=edif.crs) #rectángulo envolvente

Para seleccionar sólo los grifos en el área de interés, recortamos el dataset usando [clip](https://geopandas.org/en/stable/docs/reference/api/geopandas.clip.html).

In [ ]:
grifos=grifos.clip(env)

### 3.4 Gráfico.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

crs='3857'

vina.to_crs(crs).boundary.plot(ax=ax,color='navy',lw=2,label='Comuna de Viña del Mar')
manz.to_crs(crs).boundary.plot(ax=ax,lw=0.2,label='Manzanas')
cv.to_crs(crs).plot(ax=ax,color='r',label='Incendio')
env.to_crs(crs).boundary.plot(ax=ax,color='k',label='Envolvente')


ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

crs='3857'
edif.to_crs(crs).plot(ax=ax,color='g',label='Catastro')
damaged.to_crs(crs).plot(ax=ax,color='r',label='Dañadas')
grifos.to_crs(crs).plot(ax=ax,color='k',label='Grifos',markersize=10)

cv.to_crs(crs).boundary.plot(ax=ax,color='r',label='Convex hull')
env.to_crs(crs).boundary.plot(ax=ax,color='k',label='Envelope')


ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.legend();
plt.show()

### 3.5 Mapa interactivo con Folium

[Folium](https://python-visualization.github.io/folium/latest/) es una librería de Python que permite crear mapas interactivos de forma sencilla utilizando [Leaflet.js](https://leafletjs.com/) como motor de visualización. Su mayor ventaja es que combina la facilidad de Python con la capacidad de generar mapas web dinámicos que pueden visualizarse directamente en un Jupyter Notebook, guardarse como archivos HTML o integrarse en aplicaciones web.

En ciencia de datos geoespacial, `Folium` es útil para explorar datos espaciales de forma interactiva, permitiendo hacer zoom, mover el mapa y visualizar capas superpuestas.

In [ ]:
cv.to_crs(4326).centroid

In [ ]:
import folium
m= folium.Map(location=[-33.04,-71.55], tiles="CartoDB Positron", zoom_start=14)

folium.GeoJson(grifos,marker=folium.Circle(radius=10, fill_color="gold", fill_opacity=1, color="black", weight=1)).add_to(m)
folium.GeoJson(cv,style_function=lambda x: {"color":"blue"}).add_to(m)

folium.GeoJson(burnt,style_function=lambda x: {"fillColor": "red","color":"red","weight":1}).add_to(m)
m.save("vina.html")

m

### 3.6 ¿Cuál es la distribución de distancias entre cada vivienda quemada, y el grifo más cercano?


La función [sjoin_nrearest](https://geopandas.org/en/latest/docs/reference/api/geopandas.sjoin_nearest.html) permite unir capas en base a la menor distancie entre elementos de las geoseries. En este caso, justamente queremos agregar a cada vivienda dañada, la distancia al grifo más cercano.


In [ ]:
joined_gdf = burnt.sjoin_nearest(grifos, distance_col="distance")
joined_gdf.head()

In [ ]:
joined_gdf['distance'].hist(bins=20,edgecolor='white')
plt.ylabel('Distance (m)')
plt.xlabel('N')

plt.grid(False)

In [ ]:
joined_gdf['distance'].mean() #metros